In [ ]:
import numpy as np  
import pandas as pd 
from pulp import (LpProblem, LpMinimize, LpMaximize, LpVariable, lpSum, PULP_CBC_CMD, GLPK_CMD, LpStatus, value) 
import csv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.3 MB 15.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 46 kB 3.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
df_general = pd.read_csv('C:/Users/Reill/Downloads/final_NJ_2020_State_Legislative_data.csv')
df_general = df_general.dropna()

district_num = 40
precinct_num = 6339

precinct_pop = np.array(df_general['Total_2020_TotalAdj'])
dem_pop = np.array(df_general['Dem_2020_Sen'])
rep_pop = np.array(df_general['Rep_2020_Sen'])

model = LpProblem("Assigning-Districts", LpMaximize) 
var_name = [str(i) + '_' + str(j) for j in range(1, district_num+1) for i in range(1, precinct_num+1)]
var_name.sort()

#set to 1 if precinct is put in district, 0 otherwise
dec_var_1 = LpVariable.matrix("first", var_name, cat="Binary")
assign = np.array(dec_var_1).reshape(precinct_num, district_num)

#allocating population to districts
dec_var_2 = LpVariable.matrix("second", var_name, cat="Integer", lowBound = 0)
allocate = np.array(dec_var_2).reshape(precinct_num, district_num)

#determining repub pop
dec_var_3 = LpVariable.matrix("third", var_name, cat="Integer", lowBound = 0)
demo = np.array(dec_var_3).reshape(precinct_num, district_num)

#determining dem pop
dec_var_4 = LpVariable.matrix("fourth", var_name, cat="Integer", lowBound = 0)
repub = np.array(dec_var_4).reshape(precinct_num, district_num)

Saving final_NJ_2020_State_Legislative_data.csv to final_NJ_2020_State_Legislative_data.csv
Saving precinct-data.csv to precinct-data.csv


In [ ]:
obj = lpSum(assign) #problem may lie with this...may just minimize the precincts assigned
model += obj

#makes sure that all precincts are assigned
for i in range(precinct_num):
  model += lpSum(assign[i][j] for j in range(district_num)) <= 1

for i in range(precinct_num):
  model += lpSum(allocate[i][j] for j in range(district_num)) == precinct_pop[i]
  model += lpSum(demo[i][j] for j in range(district_num)) == dem_pop[i]
  model += lpSum(repub[i][j] for j in range(district_num)) == rep_pop[i]

hold_sum = sum(precinct_pop)
#makes sure that for population to be allocated, precinct must be assigned first
for i in range(precinct_num):
  for j in range(district_num):
    model += allocate[i][j] <= hold_sum * assign[i][j]
    if assign[i][j] == 1:
      model += allocate[i][j] == assign[i][j] * precinct_pop[i]

#contiguous precinct constraint
df_contig = pd.read_csv('C:/Users/Reill/Downloads/Contiguity.csv', names=["GEOID20", "CONTIGUITY"])
df_list = df_contig['CONTIGUITY']

indexes = []
first = 0

for i in range(0,precinct_num):
  hold = df_list.iloc[i][1:-1].split(", ")
  for k in range(len(hold)):
    hold[k] = hold[k][1:-1]
    for l in range(precinct_num):
      if df_contig['GEOID20'].iloc[l] == hold[k]:
        indexes.append(l)
  if indexes != []:
    tot = assign[indexes[0]][0]  
    for j in range(0, district_num):
      for k in range(0, len(indexes)):
        if k != 0 or j != 0:
          tot += assign[indexes[k]][j] 
      model += assign[i][j] <= tot
  indexes = []

#implementation idea: for 1/5 of districts, vastly amplify number of 
# repubs in it, but for rest, make sure demos > repubs
for j in range(district_num):
  if j < district_num/5:
    model += lpSum(repub[i][j] for i in range(precinct_num)) <= 5*lpSum(demo[i][j] for i in range(precinct_num))
  else:
    model += lpSum(repub[i][j] for i in range(precinct_num)) >= lpSum(demo[i][j] for i in range(precinct_num))

average_pop = (float)(hold_sum/district_num)
#constraint to keep population of districts within +/- 10% of average
for j in range(district_num):
  model += lpSum(allocate[i][j] for i in range(precinct_num)) <= average_pop + (average_pop * .1)
  model += lpSum(allocate[i][j] for i in range(precinct_num)) >= average_pop - (average_pop * .1)


NameError: ignored

In [ ]:
#pulp.pulpTestAll()

model.solve(PULP_CBC_CMD(msg=True)) #not sure if this is the right solver...
print(LpStatus[model.status])
print(value(obj))

res_val = []
bad = 0
for i in range(precinct_num):
  for j in range(district_num):
    output = {'Precinct': i, 'District': j+1, 'Assignment': int(assign[i][j].value()*(j+1)), 'Allocation': allocate[i][j].value()}
    if int(assign[i][j].value()*(j+1)) != 0:
      if int(assign[i][j].value()*(j+1)) > 40:
        bad = 1
      else:
        print(output)
    res_val.append(output)

if bad == 1:
  print("Bad Output")   